<font color = "#CC3D3D">
# DW(Data Wrangling) Competition

<font color = "#CC3D3D">
## Step 1) Make features ##
</font>    
- 상품판매 데이터로부터 적절한 파생변수(feature)를 만든 후 학습용(`X_train.csv`)과 평가용(`X_test.csv`)으로 나누어 저장한다.
- scikit-learn에서는 categorical feature를 사용할 수 없기 때문에 One-hot-encoding을 통해 numeric feature로 변경해야 한다. (아래 `[파생변수 11]` 코드 참조)

#### 판매데이터 구조
- custid: 고객ID
- sales_date: 판매일자 (문자)
- str_nm: 판매시간 (4자리 숫자)
- goodcd: 상품코드
- brd_nm: 브랜드명
- corner_nm: 코너명
- pc_nm: PC명
- part_nm: 파트명
- team_nm: 판매팀명
- buyer_nm: Buying MD
- import_flg: 수입상품여부
- tot_amt: 판매금액
- dis_amt: 할인금액
- net_amt: 판매금액 - 할인금액
- inst_mon: 할부개월수

<p>*`백화점 영업조직`: 306개 corner < 76개 pc < 29개 part < 3개 team*</p>
<p>*`y_train.csv`에서 target값 0은 남자, 1은 여자를 나타냄*</p>

In [147]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('max_columns', 30, 'max_rows', 30)

*파생변수를 만들기 위한 학습용과 제출용 상품판매 데이터를 읽고 하나로 합친다.*

In [148]:
train = pd.read_csv('train_transactions.csv', encoding='cp949')
test = pd.read_csv('test_transactions.csv', encoding='cp949').sort_values(by='custid')
tr = pd.concat([train, test]); tr

,custid,sales_date,sales_time,str_nm,goodcd,brd_nm,corner_nm,pc_nm,part_nm,team_nm,buyer_nm,import_flg,tot_amt,dis_amt,net_amt,inst_mon
0,18,2000-12-09 00:00:00,1523,신촌점,2116052008000,크리니크,수입종합화장품,화장품,패션잡화,잡화가용팀,화장품,1,43000,2150,40850,1
1,18,2000-12-09 00:00:00,1730,신촌점,4220830013076,BCBG,칼라드래디셔널,타운란제리,여성캐주얼,의류패션팀,엘레강스캐주얼,0,148000,0,148000,3
2,18,2000-12-09 00:00:00,1712,신촌점,2116052008000,크리니크,수입종합화장품,화장품,패션잡화,잡화가용팀,화장품,1,43000,4300,38700,1
3,18,2000-12-09 00:00:00,1710,신촌점,2116052008000,크리니크,수입종합화장품,화장품,패션잡화,잡화가용팀,화장품,1,-43000,-2150,-40850,1
4,18,2001-02-03 00:00:00,1822,신촌점,2116052008000,크리니크,수입종합화장품,화장품,패션잡화,잡화가용팀,화장품,1,19000,1900,17100,1
5,18,2001-02-24 00:00:00,1610,신촌점,4104881000379,서도스카프,스카프,섬유,패션잡화,잡화가용팀,섬유,0,19000,0,19000,1
6,18,2001-01-06 00:00:00,1810,신촌점,4100110001130,비비안스타킹,스타킹,섬유,패션잡화,잡화가용팀,섬유,0,13300,0,13300,1
7,18,2001-01-13 00:00:00,1710,신촌점,4207730013076,피에르가르뎅,인텔리젼스캐주얼,타운란제리,여성캐주얼,의류패션팀,엘레강스캐주얼,0,50000,0,50000,2
8,18,2001-02-10 00:00:00,1732,신촌점,4405620026000,지오다노,영캐주얼,영캐주얼,영플라자,의류패션팀,유니캐주얼,0,39800,3980,35820,1
9,18,2001-03-03 00:00:00,1522,신촌점,4140440003200,키사,싸롱화,피혁A,패션잡화,잡화가용팀,피혁A,0,148000,14800,133200,3


*파생변수를 저장할 빈 리스트를 만든다.*

In [149]:
features = []

<font color='green'>
### Numeric Features

**[파생변수 1]** 총 구매액

In [150]:
f = tr.groupby('custid')['tot_amt'].agg([('총구매액', 'sum')]).reset_index()
features.append(f); f

,custid,총구매액
0,18,680100
1,21,353450
2,23,5671400
3,26,1964000
4,35,885000
5,50,3580200
6,69,816300
7,82,752200
8,92,1145400
9,100,442160


**[파생변수 2]** 구매건수

In [151]:
f = tr.groupby('custid')['tot_amt'].agg([('구매건수', 'size')]).reset_index()
features.append(f); f

,custid,구매건수
0,18,15
1,21,9
2,23,36
3,26,28
4,35,5
5,50,36
6,69,14
7,82,5
8,92,13
9,100,16


**[파생변수 3]** 평균 구매가격

In [152]:
f = tr.groupby('custid')['tot_amt'].agg([('평균구매가격', 'mean')]).reset_index()
f.iloc[:,1] = f.iloc[:,1].apply(round)
features.append(f); f

,custid,평균구매가격
0,18,45340
1,21,39272
2,23,157539
3,26,70143
4,35,177000
5,50,99450
6,69,58307
7,82,150440
8,92,88108
9,100,27635


**[파생변수 4]** 평균 할부개월수

In [153]:
f = tr.groupby('custid')['inst_mon'].agg([('평균할부개월수', 'mean')]).reset_index()
f.iloc[:,1] = f.iloc[:,1].apply(round, args=(1,))
features.append(f); f

,custid,평균할부개월수
0,18,1.7
1,21,1.2
2,23,2.8
3,26,1.4
4,35,6.0
5,50,2.4
6,69,1.7
7,82,2.6
8,92,1.9
9,100,1.0


**[파생변수 5]** 구매상품 다양성: `구매한 서로다른 브랜드 수`

In [154]:
f = tr.groupby('custid')['brd_nm'].agg([('구매브랜드종류', lambda x: x.nunique())]).reset_index()
features.append(f); f

,custid,구매브랜드종류
0,18,9
1,21,6
2,23,22
3,26,15
4,35,5
5,50,27
6,69,10
7,82,5
8,92,7
9,100,10


**[파생변수 6]** 내점일수

In [155]:
f = tr.groupby(by = 'custid')['sales_date'].agg([('내점일수','nunique')]).reset_index()
features.append(f); f

,custid,내점일수
0,18,10
1,21,8
2,23,16
3,26,14
4,35,2
5,50,20
6,69,10
7,82,3
8,92,9
9,100,6


**[파생변수 7]** 수입상품 구매비율: `수입상품 구매건수` / `총 구매건수`

In [156]:
x = tr[tr['import_flg'] == 1].groupby('custid').size() / tr.groupby('custid').size()
f = x.reset_index().rename(columns={0: '수입상품_구매비율'}).fillna(0)
f.iloc[:,1] = (f.iloc[:,1]*100).apply(round, args=(1,))
features.append(f); f

,custid,수입상품_구매비율
0,18,26.7
1,21,11.1
2,23,5.6
3,26,39.3
4,35,0.0
5,50,0.0
6,69,7.1
7,82,60.0
8,92,23.1
9,100,25.0


**[파생변수 8]** 요일 구매패턴: `주말방문비율`

In [157]:
def fw(x):
    k = x.dayofweek
    if k <= 4 :
        return('주중_방문')
    else :
        return('주말_방문')    
    
df = tr.copy()
df = df.drop_duplicates(['custid','sales_date'])

df['week'] = pd.to_datetime(df.sales_date).apply(fw)
df = pd.pivot_table(df, index='custid', columns='week', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
df['주말방문비율'] = ((df.iloc[:,1] / (df.iloc[:,1]+df.iloc[:,2]))*100).apply(round, args=(1,))
f = df.copy().iloc[:,[0,-1]]
features.append(f); f

week,custid,주말방문비율
0,18,100.0
1,21,37.5
2,23,37.5
3,26,28.6
4,35,100.0
5,50,50.0
6,69,40.0
7,82,100.0
8,92,22.2
9,100,33.3


**[파생변수 9]** 계절별 구매건수: `Spring`(3~5)/`Summer`(6~8)/`Fall`(9-11)/`Winter`(12~2)

In [158]:
def f1(x):
    k = x.month
    if 3 <= k <= 5 :
        return('봄_구매건수')
    elif 6 <= k <= 8 :
        return('여름_구매건수')
    elif 9 <= k <= 11 :    
        return('가을_구매건수')
    else :
        return('겨울_구매건수')    
    
tr['season'] = pd.to_datetime(tr.sales_date).apply(f1)
f = pd.pivot_table(tr, index='custid', columns='season', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
features.append(f); f

season,custid,가을_구매건수,겨울_구매건수,봄_구매건수,여름_구매건수
0,18,0,9,6,0
1,21,0,5,4,0
2,23,7,17,12,0
3,26,5,8,15,0
4,35,0,0,5,0
5,50,7,7,8,14
6,69,4,4,6,0
7,82,0,0,5,0
8,92,1,0,2,10
9,100,0,5,11,0


**[파생변수 10]** 시간대별 구매건수: `Morning`(09~12)/`Afternoon`/(13~17)/`Evening`(18-20)

In [159]:
def f2(x):
    if 9 <= x <= 12 :
        return('아침_구매건수')
    elif 13 <= x <= 17 :
        return('점심_구매건수')
    else :
        return('저녁_구매건수')  # datatime 필드가 시간 형식에 맞지 않은 값을 갖는 경우 저녁시간으로 처리

tr['timeslot'] = (tr.sales_time // 100).apply(f2)
f = pd.pivot_table(tr, index='custid', columns='timeslot', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
features.append(f); f

timeslot,custid,아침_구매건수,저녁_구매건수,점심_구매건수
0,18,0,6,9
1,21,3,0,6
2,23,10,11,15
3,26,0,18,10
4,35,1,4,0
5,50,1,11,24
6,69,0,3,11
7,82,0,0,5
8,92,1,8,4
9,100,0,3,13


<font color='green'>
### Categorical Features

**[파생변수 11]** 주구매 파트: 29개 파트 중 가장 많이 구매한 곳
<font color='red'>주의) 이 변수를 만드는데 시간이 많이 소요된다.

In [160]:
f = tr.groupby('custid')['part_nm'].agg([('주구매코너', lambda x: x.value_counts().reset_index().sort_values(by=['part_nm','index'], ascending=False).iloc[0,0])]).reset_index()

f = pd.get_dummies(f, columns=['주구매코너'])  # This method performs One-hot-encoding

features.append(f); f

,custid,주구매코너_가정용품,주구매코너_가정용품파트,주구매코너_골프/유니캐쥬얼,주구매코너_공산품,주구매코너_공산품파트,주구매코너_남성의류,주구매코너_남성정장스포츠,주구매코너_로얄부띠끄,주구매코너_로얄부틱,주구매코너_명품잡화,주구매코너_생식품,주구매코너_생식품파트,주구매코너_스포츠캐주얼,주구매코너_스포츠캐쥬얼,주구매코너_아동,"주구매코너_아동,스포츠",주구매코너_아동문화,주구매코너_여성의류파트,주구매코너_여성정장,주구매코너_여성캐주얼,주구매코너_여성캐쥬얼,주구매코너_영라이브,주구매코너_영어덜트캐쥬얼,주구매코너_영캐릭터,주구매코너_영플라자,주구매코너_잡화,주구매코너_잡화파트,"주구매코너_케주얼,구두,아동",주구매코너_패션잡화
0,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,35,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
5,50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6,69,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
7,82,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
8,92,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [161]:
# 12 #

In [162]:
f = tr.groupby('custid')['dis_amt'].agg([('mean','mean')]).reset_index()
features.append(f); f

,custid,mean
0,18,2998.666667
1,21,144.444444
2,23,8455.000000
3,26,6875.000000
4,35,4100.000000
5,50,3899.722222
6,69,2047.142857
7,82,6420.000000
8,92,4405.384615
9,100,783.750000


In [163]:
# 13 #

In [164]:
tr['sales_date'] = pd.to_datetime(tr['sales_date'])

In [165]:
a = tr.groupby(by = 'custid')['sales_date'].agg([('내점일수','nunique')]).reset_index()
b = tr.groupby(by = 'custid')['tot_amt'].agg([('총구매액','sum')]).reset_index()

In [166]:
f = pd.merge(a,b,how = 'inner')
f['구매력'] = round(f['총구매액']/f['내점일수'],1)
features.append(f); f

,custid,내점일수,총구매액,구매력
0,18,10,680100,68010.0
1,21,8,353450,44181.2
2,23,16,5671400,354462.5
3,26,14,1964000,140285.7
4,35,2,885000,442500.0
5,50,20,3580200,179010.0
6,69,10,816300,81630.0
7,82,3,752200,250733.3
8,92,9,1145400,127266.7
9,100,6,442160,73693.3


In [167]:
## 14 ## 

In [168]:
f = tr.groupby('custid')['tot_amt'].agg([('구매건수', 'size')]).reset_index()
f = pd.merge(f,a,how = 'inner')
f['충동구매'] = f['구매건수']/f['내점일수']
features.append(f); f

,custid,구매건수,내점일수,충동구매
0,18,15,10,1.500000
1,21,9,8,1.125000
2,23,36,16,2.250000
3,26,28,14,2.000000
4,35,5,2,2.500000
5,50,36,20,1.800000
6,69,14,10,1.400000
7,82,5,3,1.666667
8,92,13,9,1.444444
9,100,16,6,2.666667


In [169]:
## 15 할인비율 ## 할인금액 / 총금액

In [170]:
tr.head(5)

,custid,sales_date,sales_time,str_nm,goodcd,brd_nm,corner_nm,pc_nm,part_nm,team_nm,buyer_nm,import_flg,tot_amt,dis_amt,net_amt,inst_mon,season,timeslot
0,18,2000-12-09,1523,신촌점,2116052008000,크리니크,수입종합화장품,화장품,패션잡화,잡화가용팀,화장품,1,43000,2150,40850,1,겨울_구매건수,점심_구매건수
1,18,2000-12-09,1730,신촌점,4220830013076,BCBG,칼라드래디셔널,타운란제리,여성캐주얼,의류패션팀,엘레강스캐주얼,0,148000,0,148000,3,겨울_구매건수,점심_구매건수
2,18,2000-12-09,1712,신촌점,2116052008000,크리니크,수입종합화장품,화장품,패션잡화,잡화가용팀,화장품,1,43000,4300,38700,1,겨울_구매건수,점심_구매건수
3,18,2000-12-09,1710,신촌점,2116052008000,크리니크,수입종합화장품,화장품,패션잡화,잡화가용팀,화장품,1,-43000,-2150,-40850,1,겨울_구매건수,점심_구매건수
4,18,2001-02-03,1822,신촌점,2116052008000,크리니크,수입종합화장품,화장품,패션잡화,잡화가용팀,화장품,1,19000,1900,17100,1,겨울_구매건수,저녁_구매건수


In [171]:
a = tr.groupby('custid')['tot_amt'].agg([('총구입금액','sum')]).reset_index()

In [172]:
b = tr.groupby('custid')['dis_amt'].agg([('총할인금액','sum')]).reset_index()

In [173]:
c = pd.merge(a,b,how = 'inner')
c['할인금액/총금액'] = c['총할인금액']/c['총구입금액']

In [174]:
f = c.loc[:,['custid','할인금액/총금액']]
features.append(f); f

,custid,할인금액/총금액
0,18,0.066137
1,21,0.003678
2,23,0.053669
3,26,0.098014
4,35,0.023164
5,50,0.039213
6,69,0.035110
7,82,0.042675
8,92,0.050000
9,100,0.028361


In [175]:
# 고객 방문 빈도 #

In [176]:
tr['unixtime'] = tr['sales_date'].view('int64')
csu = tr.groupby('custid')['unixtime'].agg([('unique','unique')])
abc = {}
for i in range(0,10000):
    abc[i] = np.var(csu[['unique']].iloc[i,:][0])
abc.values()
csu['고객 방문빈도'] = abc.values() 
csu.head(5)

,unique,고객 방문빈도
custid,,
18,"[976320000000000000, 981158400000000000, 98297...",1.220737e+31
21,"[977356800000000000, 977270400000000000, 97873...",1.676443e+31
23,"[975110400000000000, 977270400000000000, 97675...",2.071200e+31
26,"[972259200000000000, 972950400000000000, 97355...",2.746892e+31
35,"[987897600000000000, 987811200000000000]",1.866240e+27


In [177]:
csu = csu.reset_index()

In [178]:
f = csu.loc[:,['custid','고객 방문빈도']]
features.append(f); f

,custid,고객 방문빈도
0,18,1.220737e+31
1,21,1.676443e+31
2,23,2.071200e+31
3,26,2.746892e+31
4,35,1.866240e+27
5,50,7.603903e+31
6,69,4.631328e+31
7,82,1.000305e+30
8,92,1.345849e+31
9,100,2.491845e+30


In [179]:
## 파트별 구매 ##

In [180]:
cde = tr.groupby(['part_nm','custid'])['tot_amt'].agg([('브랜드별 구매횟수', 'size')]).reset_index()
cde

,part_nm,custid,브랜드별 구매횟수
0,가정용품,50,1
1,가정용품,103,1
2,가정용품,126,1
3,가정용품,148,1
4,가정용품,175,1
5,가정용품,186,1
6,가정용품,239,1
7,가정용품,261,2
8,가정용품,302,2
9,가정용품,304,2


In [181]:
f = pd.pivot_table(cde, values='브랜드별 구매횟수', index='custid', columns='part_nm', 
                   aggfunc= sum, fill_value=0).reset_index()
features.append(f); f

part_nm,custid,가정용품,가정용품파트,골프/유니캐쥬얼,공산품,공산품파트,남성의류,남성정장스포츠,로얄부띠끄,로얄부틱,명품잡화,생식품,생식품파트,스포츠캐주얼,스포츠캐쥬얼,아동,"아동,스포츠",아동문화,여성의류파트,여성정장,여성캐주얼,여성캐쥬얼,영라이브,영어덜트캐쥬얼,영캐릭터,영플라자,잡화,잡화파트,"케주얼,구두,아동",패션잡화
0,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,0,0,12
1,21,0,0,0,4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0
2,23,0,0,0,2,0,2,0,0,0,0,0,0,8,0,0,0,0,0,0,6,0,0,0,1,3,0,0,0,14
3,26,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,5,0,0,1,19
4,35,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,2
5,50,1,0,0,0,0,5,0,0,0,0,0,0,8,0,0,0,0,0,0,1,0,0,0,1,10,0,0,0,10
6,69,0,0,0,2,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,6,1,0,0,2,0,0,0
7,82,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,3
8,92,0,0,2,0,0,0,0,0,0,7,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0
9,100,0,0,2,2,0,0,0,0,0,4,1,0,0,0,0,5,0,0,0,1,0,0,0,0,0,0,0,0,1


In [182]:
#################

<br><br>
*아래 코드를 수행하면 생성한 모든 파생변수가 병합되고 학습용과 평가용으로 나뉘어진다.*

In [183]:
X_train = DataFrame({'custid': train.custid.unique()})
for f in features :
    X_train = pd.merge(X_train, f, how='left')
print(X_train)

X_test = DataFrame({'custid': test.custid.unique()})
for f in features :
    X_test = pd.merge(X_test, f, how='left')
print(X_test)

      custid      총구매액  구매건수  평균구매가격  평균할부개월수  구매브랜드종류  내점일수  수입상품_구매비율  \
0         18    680100    15   45340      1.7        9    10       26.7   
1         21    353450     9   39272      1.2        6     8       11.1   
2         23   5671400    36  157539      2.8       22    16        5.6   
3         26   1964000    28   70143      1.4       15    14       39.3   
4         35    885000     5  177000      6.0        5     2        0.0   
5         50   3580200    36   99450      2.4       27    20        0.0   
6         69    816300    14   58307      1.7       10    10        7.1   
7         82    752200     5  150440      2.6        5     3       60.0   
8         92   1145400    13   88108      1.9        7     9       23.1   
9        100    442160    16   27635      1.0       10     6       25.0   
10       102    590000     5  118000      3.0        4     4       20.0   
11       103    836600    15   55773      2.1       13    12       26.7   
12       126   1892400   

*아래 코드를 수행하면 학습용 데이터와 평가용 데이터가 저장된다.*
<font color='red'>
*(유의사항) 화일명을 절대 변경하지 말 것!*

In [184]:
X_train.to_csv('X_train.csv', index=False, encoding='cp949')
X_test.to_csv('X_test.csv', index=False, encoding='cp949')

<font color = "#CC3D3D">
## End